# 6 集成学习 

集成学习将多个模型融合, 取长补短变得更好. 常用的方法有 boosting.

## Adaboost




Adaboost 用同一模型在不同权重的数据上拟合, 最终将这些不同权重数据上学习得到的模型融合.

假设对于**分类问题**有数据 $(x_i,y_i)\quad (i=1,2,\dotsc,n)$, 其中 $y_i\in\left\{-1,1\right\}$.

步骤: 

1. 给每条数据初始化权重 $w_1=w_2=\dotsc=w_n=\dfrac 1n$.

2. 训练模型 $G_1$, 且利用的是带权重的损失函数
$$\mathcal L_1 = \sum_{i=1}^n w_i \mathbb I_{G_1(x_i) \neq y_i}.$$

3. 计算这些模型的带权重的错误率 (即损失函数) 以及系数 $\alpha$
$$e_1 = \mathcal L_1 =   \sum_{i=1}^n w_i \mathbb I_{G_1(x_i)\neq y_i}\quad\quad\quad 
\alpha_1 = \frac 12\log\frac{1 - e_1}{e_1}.$$

4. 更新权重
$$w_i\leftarrow \frac{\exp\left\{-\alpha_1 y_i G_1(x_i)\right\}}{\sum_{j=1}^n w_j\exp\left\{-\alpha_1 y_j G_1(x_j)\right\}}w_i$$

5. 重复 2,3,4 若干次 (假设 $M$ 次) 得到 $(G_2,e_2,\alpha_2),\dotsc,(G_M,e_M,\alpha_M)$. 最终的融合分类器结果为
$$\bar G = \sum_{m=1}^M \alpha_m G_m$$

### 前向分布算法

假设 $b(x;\theta)$ 是含参模型, $\theta$ 是参数. 定义可加模型为这些模型 (函数) 乘上一些系数的和:
$$f(x) = \sum_{m=1}^M \alpha_m b(x;\theta_m)$$

问题为求解最优的系数 $\theta_i\ (i=1,\dotsc,M)$ 与权重 $\alpha_i\ (i=1,\dotsc,M)$ 使损失函数最小:
$${\rm argmin}_{\alpha,\theta} \sum_{j=1}^n\mathcal L\left(y_j,f(x_j)\right)$$

<br>

用近似的贪心算法求解以上问题, 即从 $m=0$ 开始, 每次找到一组新的 $(\gamma_m,\theta_m)$ 使损失函数最小 $(m=1,\dotsc,M)$.

$$\left\{\begin{aligned}(\alpha_m,\theta_m) &= {\rm argmin}_{\alpha,\theta}  \sum_{j=1}^n\mathcal L\left(y_j,f_{m-1}(x_j)+\alpha b(x_j;\theta)\right)\\  f_m(x) &= f_{m-1}(x) +\alpha_m b(x;\theta_m) \end{aligned}\right.$$


<br>

**定理** 若 $\alpha>0$, $b(x;\theta)\in\{-1,1\}$, 且取指数分类损失函数 $L(y,f(x)) = e^{-yf(x)}\ (y\in\{-1,1\})$, 则 Adaboost 与 前向分步算法的更新方式等价.

**证明** 考察 $(\alpha_m,\beta_m)$ 的优化目标:
$$\begin{aligned}\mathcal L &=\sum_{j=1}^n\mathcal L\left(y_j,f_{m-1}(x_j)+\alpha b(x_j;\theta)\right)=
\sum_{j=1}^n\exp\left\{-y_j\left(f_{m-1}(x_j)+\alpha b(x_j;\theta)\right)\right\}\\
&= \sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\exp\left\{-\alpha \mathbb I_{y_j= b(x_j;\theta)}\right\}\\
&= \sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}e^{-\alpha}+(e^\alpha - e^{-\alpha})\sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\mathbb I_{y_j\neq b(x_j;\theta)}
\end{aligned}
$$

固定 $\alpha$ 时, 上式的第一项是常数, 因此, $\theta$ 需使得 $\sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\mathbb I_{y_j\neq b(x_j;\theta)}$ 最小.

再关于 $\alpha$ 求极值: 
$$\mathcal L =
e^{\alpha}\sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\mathbb I_{y_j\neq b(x_j;\theta)}+
e^{-\alpha}\sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\mathbb I_{y_j= b(x_j;\theta)}
$$
是关于 $e^\alpha$ 的对勾函数, 因此最优的 $\alpha$ 为
$$\alpha_m = \frac 12\log\frac{\sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\mathbb I_{y_j= b(x_j;\theta)}}{\sum_{j=1}^n\exp\left\{-y_j f_{m-1}(x_j) \right\}\mathbb I_{y_j\neq b(x_j;\theta)}}$$

令 $w_j = \exp\left\{-y_j f_{m-1}(x_j) \right\}$, 以及以 $w$ 为权重的损失函数 $\mathcal L = \sum_{j=1}^nw_j\mathbb I_{y_j\neq b(x_j;\theta)}$, 则 
$$\left\{\begin{aligned}\theta_m& = {\argmin}_\theta \sum_{j=1}^nw_j\mathbb I_{y_j\neq b(x_j;\theta)}={\argmin}_\theta\mathcal L\\
\alpha_m  &= \frac 12 \log\frac{1- \mathcal L}{\mathcal L}  
\end{aligned}\right.
$$

且 $w_j$ 更新为
$$w_j'= \exp\left\{-y_j f_{m}(x_j) \right\}=\exp\left\{-y_j (f_{m-1}(x_j)+\alpha_mb(x;\theta_m)) \right\} =w_j\exp\left\{-y_j\alpha_mb(x;\theta_m) \right\}$$

注: 虽然 Adaboost 中 $w_j'$ 还要全体除以一个系数, 但是容易发现 $w$ 是齐次的, 所有权重一起乘或除以相同系数不影响结果.

### 误差上界

